In [6]:
"""
MFC Thesis Simulation: Experimental (Feedback) vs Control (Bang-Bang) Group
Based on Jonathan Miller's Master's Thesis

This simulation models a multi-input microbial fuel cell system with:
- 3 Inputs: Fermented urine, winery wastewater, lysed spirulina
- State Variables: Temperature, pH, EC, Trace Metals
- Outputs: ORP, MFC Voltage

Experimental Protocol:
1. Run both groups in parallel for 1 week (10,080 minutes)
2. Day 2: Apply 10g citric acid stress test to both groups
3. Compare recovery and performance
"""

import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import Tuple, List
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# PHYSICAL CONSTANTS AND PARAMETERS
# =============================================================================

@dataclass
class SubstrateProperties:
    """Properties for each input substrate"""
    name: str
    pH: float              # Typical pH of input
    EC: float              # Electrical conductivity (mS/cm)
    COD: float             # Chemical oxygen demand (mg/g)
    # Trace metals (mg/L): Fe, Mn, Co, Ni, Cu, Zn, Se
    trace_metals: np.ndarray = field(default_factory=lambda: np.zeros(7))

# Define substrate properties based on thesis
# FULLY FERMENTED NEAT URINE
# - Urea completely hydrolyzed to ammonia -> very high pH (9.0-9.5)
# - Undiluted (neat) -> extremely high EC from salts (NH4+, Na+, K+, Cl-, PO4³⁻)
# - Most organics already consumed -> very low residual COD
FERMENTED_URINE = SubstrateProperties(
    name="Fully Fermented Neat Urine",
    pH=9.3,       # Very alkaline - urea -> ammonia complete
    EC=45.0,      # Very high EC - neat/undiluted, salt-rich
    COD=3.0,      # Very low - fully fermented, organics consumed
    trace_metals=np.array([1.5, 0.25, 0.08, 0.15, 0.12, 0.4, 0.08])  # Fe, Mn, Co, Ni, Cu, Zn, Se
)

# CONCENTRATED FULLY FERMENTED WINERY WASTEWATER
# - Fermentation converts sugars to VFAs and alcohols -> lower pH
# - Concentrated -> higher EC than typical winery wastewater
# - Fully fermented -> residual COD from recalcitrant organics, phenolics, tannins
# - Original COD 50-150 mg/g, concentrated ~3x, but fermented reduces it
WINERY_WASTEWATER = SubstrateProperties(
    name="Concentrated Fermented Winery WW",
    pH=3.5,       # Very acidic - VFA accumulation from fermentation
    EC=8.0,       # Higher EC - concentrated, organic acids dissociated
    COD=200.0,    # Concentrated but fermented - residual recalcitrant COD
    trace_metals=np.array([1.2, 0.2, 0.05, 0.1, 0.25, 0.35, 0.04])  # Some metals from grape/soil
)

# LYSED SPIRULINA (suspended in water)
# - Cell lysis releases intracellular contents
# - High protein content -> moderate pH buffering
# - Rich in minerals and trace metals
LYSED_SPIRULINA = SubstrateProperties(
    name="Lysed Spirulina",
    pH=7.2,       # Near neutral with slight alkaline tendency
    EC=12.0,      # High EC from released ions
    COD=1100.0,   # 950-1200 mg/g dry - high organic content
    trace_metals=np.array([5.0, 0.5, 0.2, 0.3, 0.2, 1.0, 0.15])  # Rich in metals
)

# Metal names for indexing
METAL_NAMES = ['Fe', 'Mn', 'Co', 'Ni', 'Cu', 'Zn', 'Se']

# Optimal trace metal ranges (mg/L) from Table 1
OPTIMAL_METALS = {
    'Fe': (1.0, 10.0),
    'Mn': (0.05, 0.5),
    'Co': (0.05, 0.3),
    'Ni': (0.05, 0.5),
    'Cu': (0.05, 0.3),
    'Zn': (0.1, 1.0),
    'Se': (0.05, 0.2)
}

# Decay rate constants (per minute) for trace metals
METAL_DECAY_RATES = np.array([0.0005, 0.0003, 0.0004, 0.0004, 0.0006, 0.0005, 0.0003])

# =============================================================================
# STATE CLASS
# =============================================================================

@dataclass
class MFCState:
    """Complete state of the MFC system"""
    temperature: float = 25.0      # °C (ambient, treated as disturbance)
    pH: float = 7.0                # pH units
    EC: float = 7.0                # mS/cm
    trace_metals: np.ndarray = field(default_factory=lambda: np.array([3.0, 0.2, 0.1, 0.15, 0.1, 0.4, 0.1]))
    volume: float = 1000.0         # mL (reactor volume)
    biomass: float = 1.0           # Normalized biomass concentration
    VFA: float = 0.1               # Volatile fatty acid concentration (normalized)

    # Reference values at 25°C
    pH_25: float = 7.0
    EC_25: float = 7.0
    ORP_25: float = -200.0

    def copy(self):
        return MFCState(
            temperature=self.temperature,
            pH=self.pH,
            EC=self.EC,
            trace_metals=self.trace_metals.copy(),
            volume=self.volume,
            biomass=self.biomass,
            VFA=self.VFA,
            pH_25=self.pH_25,
            EC_25=self.EC_25,
            ORP_25=self.ORP_25
        )

# =============================================================================
# PHYSICS-BASED EQUATIONS FROM THE PDF
# =============================================================================

def temperature_correction_pH(pH_25: float, T: float, alpha: float = 0.015) -> float:
    """
    Eq: pH(T) ≈ pH_25 - α(T - 25)
    α ≈ 0.015 pH units/°C
    """
    return pH_25 - alpha * (T - 25)

def temperature_correction_EC(EC_25: float, T: float, gamma: float = 0.02) -> float:
    """
    Eq: EC(T) ≈ EC_25 [1 + γ(T - 25)]
    γ ≈ 0.02 °C⁻¹
    """
    return EC_25 * (1 + gamma * (T - 25))

def temperature_correction_ORP(ORP_25: float, T: float, beta: float = 1.5) -> float:
    """
    Eq: ORP(T) ≈ ORP_25 - β(T - 25)
    β ≈ 1.5 mV/°C
    """
    return ORP_25 - beta * (T - 25)

def calculate_ORP(pH: float, EC: float) -> float:
    """
    ORP for anaerobic systems should be negative (-300 to -100 mV)
    Modified from Eq (5): ORP ≈ -200 - 59·(pH - 7) + 5·ln(EC) + ε
    At pH 7, EC 7: ORP ≈ -200 mV (typical anaerobic)
    """
    ORP = -200 - 59 * (pH - 7.0) + 5 * np.log(max(EC, 0.1))
    noise = np.random.normal(0, 5)  # Small measurement noise
    return ORP + noise

def calculate_MFC_voltage(ORP: float, T: float, COD: float, pH: float,
                          total_metals: float, ORP_25: float = -200.0) -> float:
    """
    Combined MFC voltage model from Eq (7) and Yang et al. regression:

    V_OC(T, ORP) ≈ V_OC,25°C + 0.80(T - 25) + 0.02(ORP - ORP_25)

    Voltage is highest when pH is in optimal range (6.8-7.2)
    Voltage drops when pH is too low or too high (methanogen stress)
    """
    # Base voltage from COD and trace metals
    V_base = 150 + 8.0 * np.log(COD + 1)
    V_base += 2.0 * total_metals - 0.1 * total_metals**2

    # pH factor - optimal at pH 7.0, drops off at extremes
    # Methanogens are most active at pH 6.8-7.2
    pH_optimum = 7.0
    pH_penalty = 20.0 * (pH - pH_optimum)**2  # Quadratic penalty for deviation
    V_base -= pH_penalty

    # Temperature and ORP correction
    V_OC_25 = 200  # Base open circuit voltage at 25°C (mV)
    V_temp_corr = 0.80 * (T - 25) + 0.02 * (ORP - ORP_25)

    # Combined voltage
    V_combined = 0.5 * V_base + 0.5 * (V_OC_25 + V_temp_corr)

    # Add measurement noise
    noise = np.random.normal(0, 5)
    return max(0, V_combined + noise)

def pH_inhibition_factor(pH: float, K_I_low: float = 1e-6, K_I_high: float = 1e-8) -> float:
    """
    Eq (10): I(pH) = 1 / (1 + [H⁺]/K_I,low + K_I,high/[H⁺])
    Captures sensitivity of methanogens to pH extremes
    """
    H_conc = 10**(-pH)
    return 1.0 / (1.0 + H_conc/K_I_low + K_I_high/H_conc)

def pH_EC_coupling_strength(ORP: float, pH: float, EC_0: float, kappa: float = 0.002) -> float:
    """
    Eq (3): β = κ·EC_0 if ORP > -100 mV AND pH < 6.8, else 0
    During acidification, VFA accumulates and coupling becomes significant
    Reduced kappa from 0.05 to 0.002 for realistic coupling
    """
    if ORP > -100 and pH < 6.8:
        return kappa * EC_0
    return 0.0

# =============================================================================
# DYNAMICS AND STATE EVOLUTION
# =============================================================================

def update_temperature(T_current: float, dt: float = 1.0) -> float:
    """
    Temperature evolves as a random walk (disturbance)
    Typical diurnal variation in lab environment
    """
    # Slow drift + noise
    drift = 0.001 * np.sin(2 * np.pi * dt / (24 * 60))  # Diurnal cycle
    noise = np.random.normal(0, 0.05)
    T_new = T_current + drift + noise
    # Clamp to reasonable range
    return np.clip(T_new, 20.0, 35.0)

def update_trace_metals(metals: np.ndarray, input_metals: np.ndarray,
                        Q: float, V: float, dt: float = 1.0) -> np.ndarray:
    """
    Eq (8) for trace metals with decay:
    d[M]/dt = (Q/V)(M_in - M) - λ_M·M
    """
    new_metals = metals.copy()
    for i in range(len(metals)):
        # Mass balance + decay
        dilution = (Q / V) * (input_metals[i] - metals[i])
        decay = METAL_DECAY_RATES[i] * metals[i]
        new_metals[i] = metals[i] + dt * (dilution - decay)
        new_metals[i] = max(0.001, new_metals[i])  # Prevent negative
    return new_metals

def update_pH(pH_current: float, input_pH: float, Q: float, V: float,
              VFA: float, biomass: float, dt: float = 1.0) -> float:
    """
    pH evolution through mass balance and VFA production/consumption
    """
    # Dilution effect
    dilution = (Q / V) * (input_pH - pH_current)

    # VFA effect (accumulation lowers pH)
    VFA_effect = -0.5 * VFA * biomass

    # Buffering capacity
    buffer_strength = 0.1
    pH_target = 7.0
    buffering = buffer_strength * (pH_target - pH_current)

    pH_new = pH_current + dt * (dilution + VFA_effect + buffering)
    return np.clip(pH_new, 4.0, 10.0)

def update_EC(EC_current: float, input_EC: float, Q: float, V: float,
              pH: float, ORP: float, dt: float = 1.0) -> float:
    """
    EC evolution with pH-EC coupling during acidification
    Winery (EC=8) can dilute high EC since it's lower than urine (45) and spirulina (12)
    """
    # Dilution/concentration effect - this naturally dilutes if input_EC < EC_current
    if Q > 0:
        dilution = (Q / V) * (input_EC - EC_current)
    else:
        dilution = 0

    # Natural EC decay (ion precipitation, uptake, diffusion out)
    decay = 0.002 * EC_current

    # pH-EC coupling from Eq (3) - only significant during acidification
    beta = pH_EC_coupling_strength(ORP, pH, EC_current)
    coupling = beta * (7.0 - pH)

    EC_new = EC_current + dt * (dilution + coupling - decay)
    return np.clip(EC_new, 1.0, 60.0)

def update_VFA(VFA_current: float, COD_input: float, Q: float, V: float,
               pH: float, biomass: float, T: float, dt: float = 1.0) -> float:
    """
    VFA dynamics based on substrate loading and methanogenic consumption
    """
    # VFA production from COD
    production_rate = 0.001 * COD_input * Q / V

    # VFA consumption by methanogens (pH-dependent)
    I_pH = pH_inhibition_factor(pH)
    consumption_rate = 0.1 * VFA_current * biomass * I_pH

    # Temperature effect (Arrhenius)
    T_factor = np.exp(0.05 * (T - 25))

    VFA_new = VFA_current + dt * (production_rate - consumption_rate * T_factor)
    return np.clip(VFA_new, 0.01, 5.0)

def update_biomass(biomass_current: float, VFA: float, pH: float,
                   metals: np.ndarray, T: float, dt: float = 1.0) -> float:
    """
    Biomass growth with Monod kinetics and inhibition factors
    """
    # Check trace metal sufficiency
    metal_factor = 1.0
    for i, name in enumerate(METAL_NAMES):
        opt_low, opt_high = OPTIMAL_METALS[name]
        if metals[i] < opt_low:
            metal_factor *= metals[i] / opt_low
        elif metals[i] > opt_high * 2:
            metal_factor *= opt_high * 2 / metals[i]
    metal_factor = np.clip(metal_factor, 0.1, 1.0)

    # pH inhibition
    I_pH = pH_inhibition_factor(pH)

    # Monod kinetics for VFA
    K_s = 0.5
    mu_max = 0.01
    growth = mu_max * (VFA / (K_s + VFA)) * I_pH * metal_factor

    # Death/maintenance
    death = 0.001

    # Temperature effect
    T_factor = np.exp(0.03 * (T - 25))

    biomass_new = biomass_current + dt * biomass_current * (growth * T_factor - death)
    return np.clip(biomass_new, 0.1, 5.0)

# =============================================================================
# CONTROL STRATEGIES
# =============================================================================

# Constant flow rate: 45 mL/L/h = 0.75 mL/min for 1L reactor
FLOW_RATE = 0.75  # mL/min

def bang_bang_controller(state: MFCState, time_step: int) -> Tuple[float, float, float]:
    """
    Bang-Bang (Baseline) Controller
    Only engages a pump when a parameter is OUT OF RANGE
    Uses simple thresholds without proportional control
    Flow rate: 45 mL/L/h = 0.75 mL/min for 1L reactor
    """
    # Check what's out of range and dose accordingly

    # pH too low - dose urine (alkaline)
    if state.pH < 6.8:
        return (FLOW_RATE, 0.0, 0.0)

    # pH too high - dose winery (acidic)
    if state.pH > 7.2:
        return (0.0, FLOW_RATE, 0.0)

    # EC too low - dose spirulina (adds conductivity + nutrients)
    if state.EC < 5.0:
        return (0.0, 0.0, FLOW_RATE)

    # EC too high - dose winery (lowest EC substrate for dilution)
    if state.EC > 10.0:
        return (0.0, FLOW_RATE, 0.0)

    # Trace metals low - dose spirulina
    metal_sum = np.sum(state.trace_metals)
    if metal_sum < 2.0:
        return (0.0, 0.0, FLOW_RATE)

    # Everything in range - no dosing
    return (0.0, 0.0, 0.0)

def feedback_controller(state: MFCState, time_step: int) -> Tuple[float, float, float]:
    """
    Experimental Feedback Controller
    Delivers constant 0.75 mL/min from ONE pump selected by feedback logic
    Flow rate: 45 mL/L/h = 0.75 mL/min for 1L reactor

    Substrate effects:
    - Fully fermented neat urine: pH↑↑ (9.3), EC↑↑↑ (45), COD↓ (3)
    - Concentrated fermented winery: pH↓↓ (3.5), EC↑ (8), COD↑ (200) - lowest EC
    - Lysed spirulina: pH~ (7.2), EC↑↑ (12), COD↑↑↑ (1100), Metals↑↑↑

    Target ranges:
    - pH: 6.8 - 7.2
    - EC: 5.0 - 10.0 mS/cm
    - ORP: -300 to -100 mV (anaerobic)
    - Trace metals: 2-5 mg/L total
    """
    # Target setpoints
    pH_low = 6.8
    pH_high = 7.2
    EC_target = 7.0  # Target EC

    # Check trace metal status
    metal_sum = np.sum(state.trace_metals)
    metal_low = metal_sum < 2.5
    metal_critical = metal_sum < 1.5

    # Priority-based decision: only ONE pump per minute at constant flow
    # KEY: Be very careful with urine (EC=45) - only use when EC is low!

    # PRIORITY 1: EC elevated (>8) - use winery (EC=8) to dilute or stabilize
    if state.EC > 8.0:
        return (0.0, FLOW_RATE, 0.0)  # Winery only

    # PRIORITY 2: pH too low AND EC is low (<6) - can safely use urine
    if state.pH < 6.5 and state.EC < 6.0:
        return (FLOW_RATE, 0.0, 0.0)  # Urine only

    # PRIORITY 3: pH too low BUT EC moderate (6-8) - use spirulina (gentler)
    if state.pH < 6.5:
        return (0.0, 0.0, FLOW_RATE)  # Spirulina - pH 7.2, EC 12

    # PRIORITY 4: pH too high - need acid (winery)
    if state.pH > 7.5:
        return (0.0, FLOW_RATE, 0.0)  # Winery only

    # PRIORITY 5: Trace metals critical - need spirulina
    if metal_critical:
        return (0.0, 0.0, FLOW_RATE)  # Spirulina only

    # PRIORITY 6: Trace metals low - need spirulina
    if metal_low:
        return (0.0, 0.0, FLOW_RATE)  # Spirulina only

    # PRIORITY 7: pH slightly low (6.5-6.8) and EC low - urine OK
    if state.pH < pH_low and state.EC < 6.0:
        return (FLOW_RATE, 0.0, 0.0)  # Urine only

    # PRIORITY 8: pH slightly low but EC not low - spirulina
    if state.pH < pH_low:
        return (0.0, 0.0, FLOW_RATE)  # Spirulina

    # PRIORITY 9: pH slightly high (7.2-7.5) - winery
    if state.pH > pH_high:
        return (0.0, FLOW_RATE, 0.0)  # Winery only

    # PRIORITY 10: Maintenance - spirulina for nutrients
    return (0.0, 0.0, FLOW_RATE)  # Spirulina

# =============================================================================
# SIMULATION ENGINE
# =============================================================================

def apply_citric_acid_perturbation(state: MFCState, mass_grams: float = 10.0):
    """
    Apply citric acid stress test
    Citric acid: pKa1=3.13, pKa2=4.76, pKa3=6.40
    10g citric acid (MW=192) in 1L = ~52 mM
    """
    # Citric acid drops pH significantly
    # 10g citric acid in 1L drops pH by approximately 2-3 units
    pH_drop = 2.5 * (mass_grams / 10.0) * (1000.0 / state.volume)
    state.pH = max(4.0, state.pH - pH_drop)

    # EC increases modestly due to citrate ions (3 carboxyl groups)
    # ~52 mM × 3 charges = ~156 mEq/L, but buffering reduces this
    # Realistic EC increase is about 2-4 mS/cm for 10g/L citric acid
    EC_increase = 3.0 * (mass_grams / 10.0) * (1000.0 / state.volume)
    state.EC = state.EC + EC_increase  # Don't cap it artificially

    # VFA spike (citric acid acts as substrate)
    state.VFA = min(5.0, state.VFA + 1.0)

def simulate_step(state: MFCState, doses: Tuple[float, float, float],
                  dt: float = 1.0) -> Tuple[MFCState, float, float]:
    """
    Simulate one time step (1 minute)
    Returns updated state and outputs (ORP, MFC voltage)
    """
    dose_urine, dose_winery, dose_spirulina = doses

    # Calculate total flow and weighted inputs
    Q_total = dose_urine + dose_winery + dose_spirulina

    if Q_total > 0:
        # Weighted average of input properties
        w_u = dose_urine / Q_total
        w_w = dose_winery / Q_total
        w_s = dose_spirulina / Q_total

        input_pH = w_u * FERMENTED_URINE.pH + w_w * WINERY_WASTEWATER.pH + w_s * LYSED_SPIRULINA.pH
        input_EC = w_u * FERMENTED_URINE.EC + w_w * WINERY_WASTEWATER.EC + w_s * LYSED_SPIRULINA.EC
        input_COD = w_u * FERMENTED_URINE.COD + w_w * WINERY_WASTEWATER.COD + w_s * LYSED_SPIRULINA.COD
        input_metals = (w_u * FERMENTED_URINE.trace_metals +
                       w_w * WINERY_WASTEWATER.trace_metals +
                       w_s * LYSED_SPIRULINA.trace_metals)
    else:
        # No input
        input_pH = state.pH
        input_EC = state.EC
        input_COD = 0
        input_metals = np.zeros(7)

    # Calculate current ORP for coupling calculations
    ORP_current = calculate_ORP(state.pH, state.EC)

    # Update states
    new_state = state.copy()

    # Temperature (disturbance)
    new_state.temperature = update_temperature(state.temperature, dt)

    # VFA dynamics
    new_state.VFA = update_VFA(state.VFA, input_COD, Q_total, state.volume,
                               state.pH, state.biomass, state.temperature, dt)

    # pH with temperature correction
    new_state.pH = update_pH(state.pH, input_pH, Q_total, state.volume,
                             state.VFA, state.biomass, dt)
    new_state.pH = temperature_correction_pH(new_state.pH, new_state.temperature)

    # EC with temperature correction
    new_state.EC = update_EC(state.EC, input_EC, Q_total, state.volume,
                             state.pH, ORP_current, dt)
    new_state.EC = temperature_correction_EC(new_state.EC, new_state.temperature)

    # Trace metals
    new_state.trace_metals = update_trace_metals(state.trace_metals, input_metals,
                                                  Q_total, state.volume, dt)

    # Biomass
    new_state.biomass = update_biomass(state.biomass, state.VFA, state.pH,
                                        state.trace_metals, state.temperature, dt)

    # Calculate outputs
    ORP = calculate_ORP(new_state.pH, new_state.EC)
    ORP = temperature_correction_ORP(ORP, new_state.temperature)

    total_metals = np.sum(new_state.trace_metals)
    # Estimate COD based on VFA and input
    effective_COD = state.VFA * 200 + input_COD * Q_total / 100

    MFC_voltage = calculate_MFC_voltage(ORP, new_state.temperature, effective_COD,
                                         new_state.pH, total_metals)

    return new_state, ORP, MFC_voltage

def run_simulation(duration_minutes: int = 10080,  # 1 week
                   perturbation_time: int = 2880,   # Day 2 (48 hours)
                   perturbation_mass: float = 10.0,
                   print_interval: int = 1) -> dict:
    """
    Run full simulation for both control and experimental groups
    Prints Arduino-style serial monitor output
    """
    print("=" * 60)
    print("MFC ARDUINO SERIAL MONITOR - DUAL REACTOR SYSTEM")
    print("=" * 60)
    print("Initializing sensors...")
    print("pH Sensor: OK")
    print("EC Sensor: OK")
    print("ORP Sensor: OK")
    print("Temp Sensor: OK")
    print("MFC Voltage Monitor: OK")
    print("Peristaltic Pumps: OK")
    print("=" * 60)
    print()

    # Initialize states for both groups
    control_state = MFCState()
    experimental_state = MFCState()

    # Data storage
    results = {
        'time': [],
        'control': {
            'pH': [], 'EC': [], 'ORP': [], 'voltage': [],
            'temperature': [], 'VFA': [], 'biomass': [],
            'trace_metals': [], 'doses': []
        },
        'experimental': {
            'pH': [], 'EC': [], 'ORP': [], 'voltage': [],
            'temperature': [], 'VFA': [], 'biomass': [],
            'trace_metals': [], 'doses': []
        }
    }

    # Run simulation
    for t in range(duration_minutes):
        # Store time
        results['time'].append(t)

        # Apply perturbation at specified time
        if t == perturbation_time:
            print()
            print("!" * 60)
            print("! WARNING: MANUAL PERTURBATION DETECTED")
            print(f"! Citric acid dose: {perturbation_mass}g added to both reactors")
            print("!" * 60)
            apply_citric_acid_perturbation(control_state, perturbation_mass)
            apply_citric_acid_perturbation(experimental_state, perturbation_mass)
            print()

        # Get control actions
        control_doses = bang_bang_controller(control_state, t)
        experimental_doses = feedback_controller(experimental_state, t)

        # Simulate one step
        control_state, control_ORP, control_V = simulate_step(control_state, control_doses)
        experimental_state, exp_ORP, exp_V = simulate_step(experimental_state, experimental_doses)

        # Store results - Control
        results['control']['pH'].append(control_state.pH)
        results['control']['EC'].append(control_state.EC)
        results['control']['ORP'].append(control_ORP)
        results['control']['voltage'].append(control_V)
        results['control']['temperature'].append(control_state.temperature)
        results['control']['VFA'].append(control_state.VFA)
        results['control']['biomass'].append(control_state.biomass)
        results['control']['trace_metals'].append(control_state.trace_metals.copy())
        results['control']['doses'].append(control_doses)

        # Store results - Experimental
        results['experimental']['pH'].append(experimental_state.pH)
        results['experimental']['EC'].append(experimental_state.EC)
        results['experimental']['ORP'].append(exp_ORP)
        results['experimental']['voltage'].append(exp_V)
        results['experimental']['temperature'].append(experimental_state.temperature)
        results['experimental']['VFA'].append(experimental_state.VFA)
        results['experimental']['biomass'].append(experimental_state.biomass)
        results['experimental']['trace_metals'].append(experimental_state.trace_metals.copy())
        results['experimental']['doses'].append(experimental_doses)

        # Arduino-style serial output every print_interval minutes
        if t % print_interval == 0:
            # Calculate timestamp
            days = t // 1440
            hours = (t % 1440) // 60
            mins = t % 60

            # Print Arduino-style output
            print(f"[{days:02d}d {hours:02d}h {mins:02d}m] t={t}")
            print(f"  CTRL: pH={control_state.pH:.2f} EC={control_state.EC:.2f} ORP={control_ORP:.0f}mV T={control_state.temperature:.1f}C V={control_V:.0f}mV")
            print(f"  CTRL_DOSE: urine={control_doses[0]:.1f}mL wine={control_doses[1]:.1f}mL spir={control_doses[2]:.1f}mL")
            print(f"  EXP:  pH={experimental_state.pH:.2f} EC={experimental_state.EC:.2f} ORP={exp_ORP:.0f}mV T={experimental_state.temperature:.1f}C V={exp_V:.0f}mV")
            print(f"  EXP_DOSE:  urine={experimental_doses[0]:.1f}mL wine={experimental_doses[1]:.1f}mL spir={experimental_doses[2]:.1f}mL")

            # Status flags
            ctrl_status = "OK" if 6.5 <= control_state.pH <= 7.5 else "WARN"
            exp_status = "OK" if 6.5 <= experimental_state.pH <= 7.5 else "WARN"
            print(f"  STATUS: CTRL={ctrl_status} EXP={exp_status}")
            print()

    return results

# =============================================================================
# VISUALIZATION
# =============================================================================

def plot_results(results: dict, save_path: str = None):
    """Generate comprehensive plots of simulation results"""

    time_hours = np.array(results['time']) / 60
    time_days = time_hours / 24
    perturbation_day = 2  # Day 2

    fig = plt.figure(figsize=(16, 14))

    # Color scheme
    control_color = '#E74C3C'      # Red for control
    exp_color = '#2ECC71'          # Green for experimental
    target_color = '#3498DB'       # Blue for targets

    # 1. pH over time
    ax1 = plt.subplot(3, 2, 1)
    ax1.plot(time_days, results['control']['pH'], color=control_color,
             label='Control (Bang-Bang)', alpha=0.8, linewidth=1.5)
    ax1.plot(time_days, results['experimental']['pH'], color=exp_color,
             label='Experimental (Feedback)', alpha=0.8, linewidth=1.5)
    ax1.axhline(y=7.0, color=target_color, linestyle='--', alpha=0.5, label='Target pH')
    ax1.axhspan(6.8, 7.2, alpha=0.1, color=target_color, label='Optimal Range')
    ax1.axvline(x=perturbation_day, color='orange', linestyle=':', linewidth=2, label='Citric Acid')
    ax1.set_xlabel('Time (days)')
    ax1.set_ylabel('pH')
    ax1.set_title('pH Evolution', fontweight='bold')
    ax1.legend(loc='best', fontsize=8)
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(4, 9)

    # 2. EC over time
    ax2 = plt.subplot(3, 2, 2)
    ax2.plot(time_days, results['control']['EC'], color=control_color,
             label='Control', alpha=0.8, linewidth=1.5)
    ax2.plot(time_days, results['experimental']['EC'], color=exp_color,
             label='Experimental', alpha=0.8, linewidth=1.5)
    ax2.axhspan(5.0, 10.0, alpha=0.1, color=target_color, label='Optimal Range')
    ax2.axvline(x=perturbation_day, color='orange', linestyle=':', linewidth=2)
    ax2.set_xlabel('Time (days)')
    ax2.set_ylabel('EC (mS/cm)')
    ax2.set_title('Electrical Conductivity Evolution', fontweight='bold')
    ax2.legend(loc='best', fontsize=8)
    ax2.grid(True, alpha=0.3)

    # 3. ORP over time
    ax3 = plt.subplot(3, 2, 3)
    ax3.plot(time_days, results['control']['ORP'], color=control_color,
             label='Control', alpha=0.8, linewidth=1.5)
    ax3.plot(time_days, results['experimental']['ORP'], color=exp_color,
             label='Experimental', alpha=0.8, linewidth=1.5)
    ax3.axhline(y=-100, color='red', linestyle='--', alpha=0.5, label='Acidification Threshold')
    ax3.axvline(x=perturbation_day, color='orange', linestyle=':', linewidth=2)
    ax3.set_xlabel('Time (days)')
    ax3.set_ylabel('ORP (mV)')
    ax3.set_title('Oxidation-Reduction Potential', fontweight='bold')
    ax3.legend(loc='best', fontsize=8)
    ax3.grid(True, alpha=0.3)

    # 4. MFC Voltage over time
    ax4 = plt.subplot(3, 2, 4)
    ax4.plot(time_days, results['control']['voltage'], color=control_color,
             label='Control', alpha=0.8, linewidth=1.5)
    ax4.plot(time_days, results['experimental']['voltage'], color=exp_color,
             label='Experimental', alpha=0.8, linewidth=1.5)
    ax4.axvline(x=perturbation_day, color='orange', linestyle=':', linewidth=2)
    ax4.set_xlabel('Time (days)')
    ax4.set_ylabel('Voltage (mV)')
    ax4.set_title('MFC Output Voltage', fontweight='bold')
    ax4.legend(loc='best', fontsize=8)
    ax4.grid(True, alpha=0.3)

    # 5. VFA and Biomass
    ax5 = plt.subplot(3, 2, 5)
    ax5_twin = ax5.twinx()

    l1 = ax5.plot(time_days, results['control']['VFA'], color=control_color,
                  linestyle='-', alpha=0.8, label='Control VFA')
    l2 = ax5.plot(time_days, results['experimental']['VFA'], color=exp_color,
                  linestyle='-', alpha=0.8, label='Exp. VFA')
    l3 = ax5_twin.plot(time_days, results['control']['biomass'], color=control_color,
                       linestyle='--', alpha=0.6, label='Control Biomass')
    l4 = ax5_twin.plot(time_days, results['experimental']['biomass'], color=exp_color,
                       linestyle='--', alpha=0.6, label='Exp. Biomass')

    ax5.axvline(x=perturbation_day, color='orange', linestyle=':', linewidth=2)
    ax5.set_xlabel('Time (days)')
    ax5.set_ylabel('VFA (normalized)')
    ax5_twin.set_ylabel('Biomass (normalized)')
    ax5.set_title('VFA and Biomass Dynamics', fontweight='bold')

    lines = l1 + l2 + l3 + l4
    labels = [l.get_label() for l in lines]
    ax5.legend(lines, labels, loc='best', fontsize=8)
    ax5.grid(True, alpha=0.3)

    # 6. Cumulative Energy Production
    ax6 = plt.subplot(3, 2, 6)

    # Calculate cumulative energy (voltage × time)
    control_energy = np.cumsum(results['control']['voltage']) / 1000  # mWh
    exp_energy = np.cumsum(results['experimental']['voltage']) / 1000

    ax6.plot(time_days, control_energy, color=control_color,
             label='Control', linewidth=2)
    ax6.plot(time_days, exp_energy, color=exp_color,
             label='Experimental', linewidth=2)
    ax6.axvline(x=perturbation_day, color='orange', linestyle=':', linewidth=2)
    ax6.set_xlabel('Time (days)')
    ax6.set_ylabel('Cumulative Energy (mWh)')
    ax6.set_title('Cumulative Energy Production', fontweight='bold')
    ax6.legend(loc='best', fontsize=8)
    ax6.grid(True, alpha=0.3)

    # Add improvement percentage annotation
    improvement = (exp_energy[-1] - control_energy[-1]) / control_energy[-1] * 100
    ax6.annotate(f'Improvement: {improvement:.1f}%',
                 xy=(0.95, 0.05), xycoords='axes fraction',
                 ha='right', fontsize=10, fontweight='bold',
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"\nPlot saved to: {save_path}")

    return fig

def plot_recovery_analysis(results: dict, save_path: str = None):
    """Analyze recovery after citric acid perturbation"""

    time_minutes = np.array(results['time'])
    perturbation_time = 2880  # Minute 2880 = Day 2

    # Extract post-perturbation data (first 24 hours after perturbation)
    recovery_window = 1440  # 24 hours
    post_start = perturbation_time
    post_end = min(perturbation_time + recovery_window, len(time_minutes))

    post_indices = range(post_start, post_end)
    post_time_hours = (time_minutes[post_indices] - perturbation_time) / 60

    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # pH Recovery
    ax1 = axes[0, 0]
    control_pH_post = [results['control']['pH'][i] for i in post_indices]
    exp_pH_post = [results['experimental']['pH'][i] for i in post_indices]

    ax1.plot(post_time_hours, control_pH_post, 'r-', label='Control', linewidth=2)
    ax1.plot(post_time_hours, exp_pH_post, 'g-', label='Experimental', linewidth=2)
    ax1.axhline(y=6.8, color='blue', linestyle='--', alpha=0.5, label='Lower Target')
    ax1.axhline(y=7.2, color='blue', linestyle='--', alpha=0.5, label='Upper Target')
    ax1.fill_between(post_time_hours, 6.8, 7.2, alpha=0.1, color='blue')
    ax1.set_xlabel('Hours After Perturbation')
    ax1.set_ylabel('pH')
    ax1.set_title('pH Recovery After Citric Acid Stress', fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Calculate recovery time (time to reach pH > 6.5)
    control_recovery = None
    exp_recovery = None
    for i, t in enumerate(post_time_hours):
        if control_recovery is None and control_pH_post[i] > 6.5:
            control_recovery = t
        if exp_recovery is None and exp_pH_post[i] > 6.5:
            exp_recovery = t

    recovery_text = f"Recovery to pH > 6.5:\nControl: {control_recovery:.1f}h\nExperimental: {exp_recovery:.1f}h" if control_recovery and exp_recovery else ""
    ax1.annotate(recovery_text, xy=(0.95, 0.05), xycoords='axes fraction',
                 ha='right', fontsize=9, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    # Voltage Recovery
    ax2 = axes[0, 1]
    control_V_post = [results['control']['voltage'][i] for i in post_indices]
    exp_V_post = [results['experimental']['voltage'][i] for i in post_indices]

    ax2.plot(post_time_hours, control_V_post, 'r-', label='Control', linewidth=2)
    ax2.plot(post_time_hours, exp_V_post, 'g-', label='Experimental', linewidth=2)
    ax2.set_xlabel('Hours After Perturbation')
    ax2.set_ylabel('Voltage (mV)')
    ax2.set_title('Voltage Recovery After Citric Acid Stress', fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # VFA Response
    ax3 = axes[1, 0]
    control_VFA_post = [results['control']['VFA'][i] for i in post_indices]
    exp_VFA_post = [results['experimental']['VFA'][i] for i in post_indices]

    ax3.plot(post_time_hours, control_VFA_post, 'r-', label='Control', linewidth=2)
    ax3.plot(post_time_hours, exp_VFA_post, 'g-', label='Experimental', linewidth=2)
    ax3.set_xlabel('Hours After Perturbation')
    ax3.set_ylabel('VFA (normalized)')
    ax3.set_title('VFA Response to Stress', fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)

    # Summary Statistics
    ax4 = axes[1, 1]
    ax4.axis('off')

    # Calculate statistics
    control_avg_V = np.mean(results['control']['voltage'])
    exp_avg_V = np.mean(results['experimental']['voltage'])
    control_total_E = np.sum(results['control']['voltage']) / 1000
    exp_total_E = np.sum(results['experimental']['voltage']) / 1000

    # pH time in optimal range
    control_pH_optimal = sum(1 for p in results['control']['pH'] if 6.8 <= p <= 7.2)
    exp_pH_optimal = sum(1 for p in results['experimental']['pH'] if 6.8 <= p <= 7.2)
    total_time = len(results['control']['pH'])

    summary_text = f"""
    SIMULATION SUMMARY
    ══════════════════════════════════════

    Average Voltage:
      Control:      {control_avg_V:.1f} mV
      Experimental: {exp_avg_V:.1f} mV
      Improvement:  {(exp_avg_V - control_avg_V) / control_avg_V * 100:.1f}%

    Total Energy Production:
      Control:      {control_total_E:.1f} mWh
      Experimental: {exp_total_E:.1f} mWh
      Improvement:  {(exp_total_E - control_total_E) / control_total_E * 100:.1f}%

    Time in Optimal pH Range (6.8-7.2):
      Control:      {control_pH_optimal / total_time * 100:.1f}%
      Experimental: {exp_pH_optimal / total_time * 100:.1f}%

    Recovery Performance (pH > 6.5):
      Control:      {control_recovery:.1f} hours
      Experimental: {exp_recovery:.1f} hours
      Faster by:    {control_recovery - exp_recovery:.1f} hours
    """

    ax4.text(0.1, 0.9, summary_text, transform=ax4.transAxes,
             fontsize=11, verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"\nRecovery analysis saved to: {save_path}")

    return fig

# =============================================================================
# MAIN EXECUTION
# =============================================================================

if __name__ == "__main__":
    # Set random seed for reproducibility
    np.random.seed(123)  # Changed seed

    # Run simulation with Arduino-style output
    print("\nStarting MFC simulation...\n")

    results = run_simulation(
        duration_minutes=2880,     # 2 days (48 hours)
        perturbation_time=1440,    # Perturbation at day 1
        perturbation_mass=10.0,    # 10g citric acid
        print_interval=1           # Print every minute
    )

    # Calculate summary statistics
    ctrl_total_power = sum(results['control']['voltage']) / 1000  # mWh
    exp_total_power = sum(results['experimental']['voltage']) / 1000

    ctrl_final_metals = results['control']['trace_metals'][-1]
    exp_final_metals = results['experimental']['trace_metals'][-1]

    ctrl_total_metals = sum(ctrl_final_metals)
    exp_total_metals = sum(exp_final_metals)

    # Calculate pump engagement and substrate usage
    ctrl_urine_count = sum(1 for d in results['control']['doses'] if d[0] > 0)
    ctrl_wine_count = sum(1 for d in results['control']['doses'] if d[1] > 0)
    ctrl_spir_count = sum(1 for d in results['control']['doses'] if d[2] > 0)

    exp_urine_count = sum(1 for d in results['experimental']['doses'] if d[0] > 0)
    exp_wine_count = sum(1 for d in results['experimental']['doses'] if d[1] > 0)
    exp_spir_count = sum(1 for d in results['experimental']['doses'] if d[2] > 0)

    ctrl_urine_total = sum(d[0] for d in results['control']['doses'])
    ctrl_wine_total = sum(d[1] for d in results['control']['doses'])
    ctrl_spir_total = sum(d[2] for d in results['control']['doses'])

    exp_urine_total = sum(d[0] for d in results['experimental']['doses'])
    exp_wine_total = sum(d[1] for d in results['experimental']['doses'])
    exp_spir_total = sum(d[2] for d in results['experimental']['doses'])

    # Print summary table
    print()
    print("=" * 70)
    print("SUMMARY TABLE - 2 DAY EXPERIMENT")
    print("=" * 70)
    print()
    print("+----------------------+----------------+----------------+----------------+")
    print("|      PARAMETER       |    CONTROL     |  EXPERIMENTAL  |  TARGET RANGE  |")
    print("+----------------------+----------------+----------------+----------------+")
    print(f"| Total Power (mWh)    | {ctrl_total_power:14.2f} | {exp_total_power:14.2f} |       N/A      |")
    print(f"| Total Trace Metals   | {ctrl_total_metals:14.2f} | {exp_total_metals:14.2f} |    2.0 - 5.0   |")
    print(f"| Final pH             | {results['control']['pH'][-1]:14.2f} | {results['experimental']['pH'][-1]:14.2f} |    6.8 - 7.2   |")
    print(f"| Final EC (mS/cm)     | {results['control']['EC'][-1]:14.2f} | {results['experimental']['EC'][-1]:14.2f} |   5.0 - 10.0   |")
    print(f"| Final ORP (mV)       | {results['control']['ORP'][-1]:14.0f} | {results['experimental']['ORP'][-1]:14.0f} |  -300 to -100  |")
    print(f"| Final Voltage (mV)   | {results['control']['voltage'][-1]:14.0f} | {results['experimental']['voltage'][-1]:14.0f} |    > 200       |")
    print("+----------------------+----------------+----------------+----------------+")
    print()
    print("Trace Metal Breakdown (mg/L):")
    print("+-------+----------------+----------------+----------------+")
    print("| Metal |    CONTROL     |  EXPERIMENTAL  |  TARGET RANGE  |")
    print("+-------+----------------+----------------+----------------+")
    print(f"|  Fe   | {ctrl_final_metals[0]:14.3f} | {exp_final_metals[0]:14.3f} |   1.0 - 10.0   |")
    print(f"|  Mn   | {ctrl_final_metals[1]:14.3f} | {exp_final_metals[1]:14.3f} |  0.05 - 0.50   |")
    print(f"|  Co   | {ctrl_final_metals[2]:14.3f} | {exp_final_metals[2]:14.3f} |  0.05 - 0.30   |")
    print(f"|  Ni   | {ctrl_final_metals[3]:14.3f} | {exp_final_metals[3]:14.3f} |  0.05 - 0.50   |")
    print(f"|  Cu   | {ctrl_final_metals[4]:14.3f} | {exp_final_metals[4]:14.3f} |  0.05 - 0.30   |")
    print(f"|  Zn   | {ctrl_final_metals[5]:14.3f} | {exp_final_metals[5]:14.3f} |   0.1 - 1.0    |")
    print(f"|  Se   | {ctrl_final_metals[6]:14.3f} | {exp_final_metals[6]:14.3f} |  0.05 - 0.20   |")
    print("+-------+----------------+----------------+----------------+")
    print()
    print("Pump Engagement Count (times activated):")
    print("+----------------------+----------------+----------------+")
    print("|        PUMP          |    CONTROL     |  EXPERIMENTAL  |")
    print("+----------------------+----------------+----------------+")
    print(f"| Urine Pump           | {ctrl_urine_count:14d} | {exp_urine_count:14d} |")
    print(f"| Winery Pump          | {ctrl_wine_count:14d} | {exp_wine_count:14d} |")
    print(f"| Spirulina Pump       | {ctrl_spir_count:14d} | {exp_spir_count:14d} |")
    print("+----------------------+----------------+----------------+")
    print()
    print("Total Substrate Used (mL):")
    print("+----------------------+----------------+----------------+")
    print("|      SUBSTRATE       |    CONTROL     |  EXPERIMENTAL  |")
    print("+----------------------+----------------+----------------+")
    print(f"| Fermented Urine      | {ctrl_urine_total:14.1f} | {exp_urine_total:14.1f} |")
    print(f"| Fermented Winery WW  | {ctrl_wine_total:14.1f} | {exp_wine_total:14.1f} |")
    print(f"| Lysed Spirulina      | {ctrl_spir_total:14.1f} | {exp_spir_total:14.1f} |")
    print(f"| TOTAL                | {ctrl_urine_total+ctrl_wine_total+ctrl_spir_total:14.1f} | {exp_urine_total+exp_wine_total+exp_spir_total:14.1f} |")
    print("+----------------------+----------------+----------------+")
    print()
    print("=" * 70)
    print("END OF EXPERIMENT")
    print("=" * 70)

Streaming output truncated to the last 5000 lines.
[01d 12h 13m] t=2173
  CTRL: pH=6.49 EC=64.34 ORP=-155mV T=28.6C V=186mV
  CTRL_DOSE: urine=0.8mL wine=0.0mL spir=0.0mL
  EXP:  pH=6.98 EC=1.27 ORP=-202mV T=24.9C V=193mV
  EXP_DOSE:  urine=0.0mL wine=0.0mL spir=0.8mL
  STATUS: CTRL=WARN EXP=OK

[01d 12h 14m] t=2174
  CTRL: pH=6.48 EC=64.23 ORP=-148mV T=28.5C V=177mV
  CTRL_DOSE: urine=0.8mL wine=0.0mL spir=0.0mL
  EXP:  pH=6.98 EC=1.28 ORP=-194mV T=25.0C V=184mV
  EXP_DOSE:  urine=0.0mL wine=0.0mL spir=0.8mL
  STATUS: CTRL=WARN EXP=OK

[01d 12h 15m] t=2175
  CTRL: pH=6.48 EC=64.18 ORP=-152mV T=28.5C V=191mV
  CTRL_DOSE: urine=0.8mL wine=0.0mL spir=0.0mL
  EXP:  pH=6.98 EC=1.28 ORP=-203mV T=25.0C V=191mV
  EXP_DOSE:  urine=0.0mL wine=0.0mL spir=0.8mL
  STATUS: CTRL=WARN EXP=OK

[01d 12h 16m] t=2176
  CTRL: pH=6.49 EC=64.16 ORP=-160mV T=28.5C V=181mV
  CTRL_DOSE: urine=0.8mL wine=0.0mL spir=0.0mL
  EXP:  pH=6.97 EC=1.29 ORP=-206mV T=24.9C V=191mV
  EXP_DOSE:  urine=0.0mL wine=0.0mL spir